In [ ]:
import os

from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import xarray as xr

from sdm_eurec4a.visulization import set_custom_rcParams

In [ ]:
plt.style.use("default")
default_colors = set_custom_rcParams()
from matplotlib import rc

# rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
# rc('font',**{'family':'serif','serif':['Palatino']})
rc("text", usetex=False)

# THE PATH TO THE SCRIPT DIRECTORY
script_dir = os.path.abspath("/home/m/m301096/repositories/sdm-eurec4a/scripts/CLEO/initalize")
print(script_dir)

REPOSITORY_ROOT = Path(script_dir).parents[2]
print(REPOSITORY_ROOT)

fig_path = REPOSITORY_ROOT / Path("results/CLEO/initilization/fitting_psd")
fig_path.mkdir(parents=True, exist_ok=True)

/home/m/m301096/repositories/sdm-eurec4a/scripts/CLEO/initalize
/home/m/m301096/repositories/sdm-eurec4a


In [ ]:
import numpy as np
import xarray as xr
from sdm_eurec4a.reductions import x_y_flatten

da = xr.DataArray(
    np.arange(2 * 4).reshape(2, 4),
    dims=("dim_0", "dim_1"),
    coords={
        "dim_0": np.arange(2),
        "dim_1": np.arange(4),
    },
)
x, y = x_y_flatten(da, "dim_0")
print(x.shape)
print(y.shape)
print(x)
print(y)


da = xr.DataArray(
    np.arange(24).reshape(4, 3, 2),
    dims=("lon", "lat", "time"),
    coords={
        "time": np.arange(2),
        "lat": np.arange(3),
        "lon": np.arange(4),
    },
)
x, y = x_y_flatten(da, "lon")
print(x.shape)
print(y.shape)

print(x)
print(y)

(8,)
(8,)
[0 0 0 0 1 1 1 1]
[0 1 2 3 4 5 6 7]
(8,)
(8,)
[0 0 1 1 2 2 3 3]
[ 0  2  6  8 12 14 18 20]


### Load datasets

In [ ]:
# Load data
# mask_name = "cloud_mask"
# chosen_id = 1421
mask_name = "rain_mask"
chosen_id = 77

identified_clouds = xr.open_dataset(
    REPOSITORY_ROOT
    / Path(
        f"data/observation/cloud_composite/processed/identified_clouds/identified_clouds_{mask_name}.nc"
    )
)
# select only clouds which are between 800 and 1100 m
identified_clouds = identified_clouds.where(
    (identified_clouds.alt >= 800) & (identified_clouds.alt <= 1100), drop=True
)

distance_IC_DS = xr.open_dataset(
    REPOSITORY_ROOT
    / Path(f"data/observation/combined/distance/distance_dropsondes_clouds_{mask_name}.nc")
)

cloud_composite = xr.open_dataset(
    REPOSITORY_ROOT / Path("data/observation/cloud_composite/processed/cloud_composite.nc"),
    chunks={"time": 1000},
)

drop_sondes = xr.open_dataset(
    REPOSITORY_ROOT
    / Path("data/observation/dropsonde/Level_3/EUREC4A_JOANNE_Dropsonde-RD41_Level_3_v2.0.0.nc")
)
drop_sondes = drop_sondes.rename({"launch_time": "time"})
drop_sondes = drop_sondes.swap_dims({"sonde_id": "time"})
drop_sondes = drop_sondes.sortby("time")
drop_sondes = drop_sondes.chunk({"time": -1})

Chose an individual cloud to handle.
Use ``chosen_id = 77`` for the rain_mask case
Use ``chosen_id = 1421`` for the cloud_mask case

In [ ]:
# select a single cloud
ds_cloud = select_individual_cloud(identified_clouds, chosen_id)
ds_sonde = match_clouds_and_dropsondes(
    ds_cloud=ds_cloud,
    ds_sonde=drop_sondes,
    ds_distance=distance_IC_DS,
    max_temporal_distance=np.timedelta64(1, "h"),
    max_spatial_distance=100,
)

ds_cloudcomposite = match_clouds_and_cloudcomposite(
    ds_cloud=ds_cloud,
    ds_cloudcomposite=cloud_composite,
)
ds_cloudcomposite

#### Plot the distributions in linear and lognormal space

Compare Particle Size Distribution **WITH and WITHOUT normalized** by bin width

In [ ]:
style = dict(
    marker=".",
    linestyle="none",
    color="k",
    alpha=0.5,
)
fig, axs = plt.subplots(
    nrows=1, ncols=2, figsize=(10, 5), sharex=True, sharey=True, layout="constrained"
)

fig.suptitle(f"Cloud ID: {chosen_id} - Particle Size Distribution - Norm and NOT Norm")

psd = ds_cloudcomposite["particle_size_distribution"]
symlog = symlog_from_array(psd, axes=axs[0])
axs[0].plot(
    psd["diameter"],
    psd,
    **style,
)
axs[0].set_title("Counts  #/l/µm\nnormalization by bin width")
axs[0].set_xscale("log")
axs[0].set_yscale(symlog)
axs[0].set_xlabel("Diameter x [µm]")
axs[0].set_ylabel("Counts [#/l/µm]")

psd = ds_cloudcomposite["particle_size_distribution"] * ds_cloudcomposite["bin_width"]
symlog = symlog_from_array(psd, axes=axs[1])
axs[1].plot(
    psd["diameter"],
    psd,
    **style,
)
axs[1].set_title("Total Counts #/l \nwithout normalization by bin width")
axs[1].set_xscale("log")
axs[1].set_yscale(symlog)
axs[1].set_xlabel("Diameter x [µm]")
axs[1].set_ylabel("Counts [#/l]")

for ax in axs.flatten():
    ax.set_ylim(0, None)
    ax.grid(True)

In [ ]:
ds_cloudcomposite["particle_size_distribution"].attrs

{'Long_name': 'Particle Size Distribution',
 'unit': '#/L/micrometer',
 'comment': 'histogram: each bin gives the number of droplets per liter of air, normalized by the bin width (see "bw")'}

Particle Size Distribution **NOT normalized** by bin width

In [ ]:
style = dict(
    marker=".",
    linestyle="none",
    color="k",
    alpha=0.5,
)
psd = ds_cloudcomposite["particle_size_distribution"] * ds_cloudcomposite["bin_width"]
fig, axs = plt.subplots(3, 1, figsize=(10, 7), layout="constrained")

fig.suptitle(f"Cloud ID: {chosen_id} - Particle Size Distribution - different xscales")

axs[0].plot(
    psd["diameter"],
    psd,
    **style,
)
axs[0].set_xlabel("Linear of x")
axs[0].set_xlabel("Diameter x [µm]")
axs[0].set_ylabel("Counts [#/m^3]")
axs[1].plot(
    psd["diameter"],
    psd,
    **style,
)
axs[1].set_xscale("log")
axs[1].set_title("Linear of x on log10 scale")
axs[1].set_xlabel("Diameter x [µm]")
axs[1].set_ylabel("Counts [#/m^3]")

axs[2].plot(
    np.log(psd["diameter"]),
    psd,
    **style,
)
axs[2].set_title("Linear of Ln(x)")
axs[2].set_xlabel("Diameter Ln(x) [Ln(µm)]")
axs[2].set_ylabel("Counts [#/m^3]")

for ax in axs.flatten():
    ax.set_ylabel("#/l/µm (binwidth)")

In [ ]:
import warnings


def vsd_from_psd(
    ds: xr.Dataset,
    psd_name: str = "particle_size_distribution",
    psd_factor: float = 1e-3,
    scale_name: str = "diameter",
    scale_factor: float = 1e-6,
    radius_given: bool = False,
) -> xr.DataArray:
    """
    Calculate the volume size distribution from the particle size distribution.
    VSD = PSD * 4/3 * pi * r^3

    Note
    ----
    - If the diameter is given, set radius to False.
    - Make sure, the units are correct.
    - If PSD is normalized by bin width (given in .../µm), the VSD is normalized by bin width (given in .../µm).
    - For instance given are:
        - PSD in #/l (#/(1e3m^3))
        - Diameter in µm (1e-6m)
    - Then set:
        - psd_factor to 1e-3
        - scale_factor to 1e-6
        - radius_given = False

    Parameters
    ----------
    ds : xr.Dataset
        Dataset containing the particle size distribution (``psd_name``) and the diameter or radius (``scale_name``).
    psd_name : str, optional
        The name of the particle size distribution. Default is "particle_size_distribution".
        Assumed to have units of 1/m^3.
    scale_name : str, optional
        The name of the scale. Default is "diameter".
    scale_factor : float, optional
        The scale factor. Default is 1e-6.
    radius_given : bool, optional
        If set to True, it is assumed, that under ``scale_name`` the radius is stored.
        If set to False, it is assumed, that under ``scale_name`` the diameter is stored.
        Default is False.

    Returns
    -------
    xr.DataArray
        The volume size distribution.
        Make sure to check the units and otherwise set the value!

    """
    psd = ds[psd_name] * psd_factor

    if radius_given == True:
        radius = ds[scale_name] * scale_factor
    else:
        radius = 0.5 * ds[scale_name] * scale_factor

    vsd = psd * 4 / 3 * np.pi * radius**3
    vsd.attrs = dict(
        units="m^3/m^3",
        long_name="Volume size distribution",
        description="Volume size distribution calculated from the particle size distribution. VSD = PSD * 4/3 * pi * radius^3",
    )
    vsd.name = "volume_size_distribution"
    warnings.warn("units is set to m^3/m^3. Make sure to check the units and otherwise set the value!")
    return vsd

# Ideas on how to fit a normal distribution

In [ ]:
ax = plt.subplot(111)

psd = ds_cloudcomposite["particle_size_distribution"]

symlog = symlog_from_array(psd, ax)
ax.plot(
    psd.diameter,
    psd.diameter * 0,
    marker="+",
    label="with zeros",
    linestyle="None",
)
ax.set_xscale("log")

ax.legend()

### Lets make a simple test, if one can use the 2 dim array to use a a fitting.

#### Chose an individual psd

#### Use Astropy 

In [ ]:
from astropy.modeling import models, fitting

# Fit the data using a box model.
# Bounds are not really needed but included here to demonstrate usage.
# Fit the data using a Gaussian
x, y = x_y_flatten(psd, "diameter")

g_init_log = models.Gaussian1D(amplitude=10.0, mean=2, stddev=1.0)
g_init = models.Gaussian1D(amplitude=1e5, mean=1e1, stddev=1e1)
fit_g = fitting.LevMarLSQFitter()
x_new = np.log(x)
y_new = y.copy()
g_log = fit_g(g_init, x_new, y_new)
g = fit_g(g_init, x, y_new)
# # Plot the data with the best-fit model
plt.figure(figsize=(8, 5))
plt.xscale("log")
plt.yscale(symlog)
plt.plot(x, y_new, "ko")
plt.plot(x, g_log(x_new), label="Gaussian xlog")
plt.plot(x, g(x), label="Gaussian")
plt.xlabel("Diameter")
plt.ylabel("Count")
plt.legend()

print(g.parameters)
print(g.param_names)

[4.03916544e+04 8.11481775e+00 2.53643375e+00]
('amplitude', 'mean', 'stddev')


#### Use scipy curve fitting 
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html#scipy.optimize.curve_fit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


def func(x, a, b, c):
    return a * np.exp(-b * x) + c


def gaussian(x: np.ndarray, amplitude: float = 1, mean: float = 0, sigma: float = 1):
    res = amplitude * 1 / np.sqrt(2 * np.pi) / sigma * np.exp(-0.5 * ((x - mean) / sigma) ** 2)
    return res


# def gaussian_log(x : np.ndarray, amplitude : float = 1, mean : float = 0, sigma : float= 1):
#     res = amplitude * 1/np.sqrt(2*np.pi) /sigma * np.exp(-0.5 * ((np.log(x) - mean) / sigma)**2)
#     return res


def double_gaussian(
    x: np.ndarray,
    amplitude_1: float = 1,
    mean_1: float = 0,
    sigma_1: float = 1,
    amplitude_2: float = 1,
    mean_2: float = 0,
    sigma_2: float = 1,
):
    res = gaussian(x, amplitude_1, mean_1, sigma_1) + gaussian(x, amplitude_2, mean_2, sigma_2)
    return res

In [ ]:
x, y = x_y_flatten(psd, "diameter")

plt.xscale("log")
plt.yscale(symlog)
plt.plot(
    x,
    y,
    marker="+",
    linestyle="None",
)
np.log(1e-6)

-13.815510557964274

In [ ]:
x, y = x_y_flatten(psd, "diameter")

# x, y = x_y_flatten(psd.where(psd != 0, drop = True), "diameter")

xdata = np.log(x)
np.nan_to_num(xdata, copy=False, nan=0)
ydata = y.copy()
np.nan_to_num(ydata, copy=False, nan=0)
x_split = 4.5
plt.yscale(symlog)
plt.plot(
    xdata,
    ydata,
    marker="+",
    linestyle="None",
)
plt.axvline(x_split, color="k", linestyle="--")

sigma = ydata == 0
sigma = sigma.astype(float)
sigma = sigma * 1e5 + 1e-10
# sigma[ydata == 0] = 0

idx_low = np.where(xdata <= x_split)
idx_high = np.where(xdata > x_split)
xdata_low = xdata[idx_low]
ydata_low = ydata[idx_low]
xdata_high = xdata[idx_high]
ydata_high = ydata[idx_high]
sigma_low = sigma[idx_low]
sigma_high = sigma[idx_high]
plt.plot(xdata_low, sigma_low, marker="x", linestyle="None")
plt.plot(xdata_high, sigma_high, marker="x", linestyle="None")

In [ ]:
popt_low, pcov_low = curve_fit(gaussian, xdata_low, ydata_low, p0=[1e5, -11, 3], sigma=sigma_low)
popt_high, pcov_high = curve_fit(gaussian, xdata_high, ydata_high, p0=[1e-1, -8.5, 5], sigma=sigma_high)

popt = np.concatenate((popt_low, popt_high))
pcov = np.concatenate((pcov_low, pcov_high))
plt.xscale("linear")

x_new = np.arange(0.5, 9, 0.01)


plt.plot(xdata, ydata, "k.", alpha=0.5, label="data")
plt.plot(np.log(psd.diameter), psd.median(dim="time"), "k-", label="Median")
plt.plot(xdata_low, ydata_low, ".", alpha=0.5, label="data low")
plt.plot(xdata_high, ydata_high, ".", alpha=0.5, label="data high")

plt.plot(x_new, gaussian(x_new, *popt_low), "-", label="low")
plt.plot(x_new, gaussian(x_new, *popt_high), "-", label="high")
plt.plot(x_new, double_gaussian(x_new, *popt), "r--", label="total")
# plt.yscale(symlog)

plt.legend()
# plt.ylim(1e-3, 1e5)

print(popt_low)
# print(popt_high)
# print(np.linalg.cond(pcov_low))
# print(np.linalg.cond(pcov_high))

[3.48004290e+04 2.01895369e+00 3.21150244e-01]


In [ ]:
popt_low, pcov_low = curve_fit(gaussian, xdata_low, ydata_low, p0=[1e5, -11, 3], sigma=sigma_low)
popt_high, pcov_high = curve_fit(gaussian, xdata_high, ydata_high, p0=[1e-1, -8.5, 5], sigma=sigma_high)

popt = np.concatenate((popt_low, popt_high))
pcov = np.concatenate((pcov_low, pcov_high))
plt.xscale("linear")

x_new = np.arange(0.5, 9, 0.01)

x_new = np.exp(x_new)
popt_low = list(popt_to_meter(*popt_low).values())

plt.xscale("log")
plt.plot(x, ydata, "k.", alpha=0.5, label="data")
# plt.plot(psd.diameter, psd, 'k.', alpha = 0.1)
plt.plot(psd.diameter, psd.median(dim="time"), "k-", label="Median")
plt.plot(x[idx_low], ydata_low, ".", alpha=0.5, label="data low")
plt.plot(x[idx_high], ydata_high, ".", alpha=0.5, label="data high")

plt.plot(x_new, gaussian(x_new, *popt_low), "-", label="low")
# plt.plot(x_new, gaussian(x_new, *popt_high), '-', label = 'high')
# plt.plot(x_new, double_gaussian(x_new, *popt), 'r--', label = 'total')
# plt.yscale(symlog)

plt.legend()
# plt.ylim(1e-3, 1e5)

print(popt_low)
# print(popt_high)
# print(np.linalg.cond(pcov_low))
# print(np.linalg.cond(pcov_high))

[34800.42903134127, 7.530441621654451, 1.378712708056673]


In [ ]:
def popt_to_meter(amplitude, mean, sigma):
    """
    Converts the units of amplitude, mean and sigma to the correct type.

    Parameters
    ----------
    amplitude : float
        The amplitude of the gaussian.
        It is assumed to be given in #/l thus #/dm^3
    mean : float
        The mean of the gaussian.
        It is assumed to be given in ln(m).
    sigma : float
        The standard deviation of the gaussian.
        It is assumed to be given in ln(m).

    Returns
    -------
    dict
        A dictionary containing the amplitude, mean and sigma in the correct units.
        The amplitude is given in #/cm^3, the mean and sigma are given in m.
    """
    amplitude = amplitude
    mean = np.exp(mean)
    sigma = np.exp(sigma)
    return dict(
        amplitude=amplitude,
        mean=mean,
        sigma=sigma,
    )


list(popt_to_meter(*popt_low).values())

# plt.plot(x_new, gaussian(x_new, *popt_high), '-', label = 'high')

[inf, 1863.9284729985998, 3.9697880606869833]

Double gaussian curve fit

In [ ]:
xdata = np.linspace(0, 15, 100)
y1 = gaussian(xdata, 2.5, 1.3, 0.5)
y2 = gaussian(xdata, 2.5, 10.3, 1.5)
y = y1 + y2
rng = np.random.default_rng()
y_noise = 0.01 * rng.normal(size=xdata.size)
ydata = y + y_noise

plt.plot(xdata, ydata, "b-", label="data")
popt, pcov = curve_fit(gaussian, xdata, ydata, p0=[11, 1, 1])
plt.plot(xdata, gaussian(xdata, *popt), "r-")
x_split = 5
idx_low = np.where(xdata > x_split)
idx_high = np.where(xdata <= x_split)
xdata_low = xdata[idx_low]
ydata_low = ydata[idx_low]
xdata_high = xdata[idx_high]
ydata_high = ydata[idx_high]

popt_low, pcov_low = curve_fit(gaussian, xdata_low, ydata_low, p0=[11, 1, 1])
popt_high, pcov_high = curve_fit(gaussian, xdata_high, ydata_high, p0=[11, 1, 1])

popt = np.concatenate((popt_low, popt_high))

plt.plot(xdata, ydata, "b-", label="data")
plt.plot(xdata, double_gaussian(xdata, *popt), "r-")

In [ ]:
xdata = np.linspace(0, 15, 100)

y1 = gaussian(xdata, 2.5, 1.3, 0.5)
y2 = gaussian(xdata, 2.5, 10.3, 1.5)
y = y1 + y2
rng = np.random.default_rng()
y_noise = 0.01 * rng.normal(size=xdata.size)
ydata = y + y_noise
ydata = np.exp(ydata)


x_split = 5
idx_low = np.where(xdata > x_split)
idx_high = np.where(xdata <= x_split)
xdata_low = xdata[idx_low]
ydata_low = ydata[idx_low]
xdata_high = xdata[idx_high]
ydata_high = ydata[idx_high]

popt_low, pcov_low = curve_fit(gaussian, xdata_low, ydata_low, p0=[11, 1, 1])
popt_high, pcov_high = curve_fit(gaussian, xdata_high, ydata_high, p0=[11, 1, 1])

popt = np.concatenate((popt_low, popt_high))


plt.plot(xdata, ydata, "b-", label="data")
plt.plot(xdata, ydata, "b-", label="data")
plt.plot(xdata, double_gaussian(xdata, *popt), "r-")